# Bug Prediction Demo

Interactive demonstration of the bug prediction model.

**Contents:**
1. Load the trained model
2. Classic bug examples (division by zero, index errors)
3. Subtle bugs (off-by-one, mutable defaults)
4. Edge cases where the model struggles
5. Interactive prediction widget

In [ ]:
# Setup
import sys
sys.path.append('..')

from src.evaluate import CodePredictor, print_prediction

# Load the predictor
predictor = CodePredictor(config_path='../config.yaml')
print('Model loaded successfully!')
print(f'Model type: {predictor.model_type}')

## Helper Function for Demo

In [ ]:
def demo_predict(code, title="", show_features=False):
    """Run prediction with nice formatting."""
    print('\n' + '='*70)
    if title:
        print(f'Example: {title}')
        print('-'*70)
    print(code)
    print('-'*70)
    
    result = predictor.predict(code)
    
    if result.get('probability') is not None:
        prob = result['probability']
        pred = result['prediction']
        conf = result.get('confidence', 0)
        
        # Color coding
        if pred == 'BUGGY':
            symbol = '\u2718'  # X mark
            status = f'\033[91mBUGGY\033[0m'
        else:
            symbol = '\u2714'  # Check mark
            status = f'\033[92mCLEAN\033[0m'
        
        print(f'Prediction: {symbol} {status}')
        print(f'Bug Probability: {prob:.1%}')
        print(f'Confidence: {conf:.1%}')
        
        if show_features and result.get('features'):
            print('\nTop Features:')
            for name, value in sorted(result['features'].items(), 
                                      key=lambda x: abs(x[1]), reverse=True)[:5]:
                if value != 0:
                    print(f'  {name}: {value}')
    else:
        print(f'Error: {result.get("error")}')
    
    print('='*70)
    return result

## 1. Classic Bug Examples

These are obvious bugs that should be easy to detect.

In [ ]:
# Division by Zero (no check)
demo_predict('''
def divide(a, b):
    return a / b
''', "Division without zero check")

In [ ]:
# Index out of bounds
demo_predict('''
def get_element(items, index):
    return items[index]
''', "Index access without bounds check")

In [ ]:
# Missing return statement
demo_predict('''
def calculate(x):
    result = x * 2
    # Forgot to return!
''', "Missing return statement")

In [ ]:
# Uninitialized variable
demo_predict('''
def process(data):
    if data:
        result = data[0]
    return result  # May be undefined!
''', "Potentially uninitialized variable")

## 2. Subtle Python Bugs

These are common Python gotchas that trip up many developers.

In [ ]:
# Mutable default argument (classic Python bug!)
demo_predict('''
def append_to_list(item, items=[]):
    items.append(item)
    return items
''', "Mutable default argument", show_features=True)

In [ ]:
# Off-by-one error
demo_predict('''
def get_last_items(items, n):
    result = []
    for i in range(n + 1):  # Bug: should be range(n)
        result.append(items[i])
    return result
''', "Off-by-one error in loop")

In [ ]:
# Bare except (bad practice)
demo_predict('''
def read_file(path):
    try:
        with open(path) as f:
            return f.read()
    except:
        pass  # Silently ignores ALL errors!
''', "Bare except clause")

In [ ]:
# Missing null check
demo_predict('''
def process_data(data):
    result = data.strip().lower()
    parts = result.split(',')
    return parts[0]
''', "Missing null check")

In [ ]:
# Using eval (security risk!)
demo_predict('''
def run_command(cmd):
    return eval(cmd)
''', "Using eval (security risk)")

## 3. Clean Code Examples

Well-written functions that should be classified as clean.

In [ ]:
# Well-structured function with validation
demo_predict('''
def calculate_average(numbers):
    """Calculate the average of a list of numbers."""
    if not numbers:
        return 0
    return sum(numbers) / len(numbers)
''', "Well-structured function with validation")

In [ ]:
# Binary search (classic algorithm)
demo_predict('''
def binary_search(arr, target):
    """Binary search for target in sorted array."""
    left, right = 0, len(arr) - 1

    while left <= right:
        mid = (left + right) // 2
        if arr[mid] == target:
            return mid
        elif arr[mid] < target:
            left = mid + 1
        else:
            right = mid - 1

    return -1
''', "Binary search implementation")

In [ ]:
# Safe division with check
demo_predict('''
def safe_divide(a, b):
    """Divide a by b, returning None if b is zero."""
    if b == 0:
        return None
    return a / b
''', "Safe division with zero check")

In [ ]:
# Fixed mutable default
demo_predict('''
def append_to_list(item, items=None):
    """Append item to list, creating new list if needed."""
    if items is None:
        items = []
    items.append(item)
    return items
''', "Fixed mutable default pattern")

## 4. Edge Cases & Model Limitations

Cases where the model may struggle or give unexpected results.

In [ ]:
# Very short function (minimal features)
demo_predict('''
def add(a, b):
    return a + b
''', "Very short function (minimal features)")

In [ ]:
# Complex but correct (deeply nested)
demo_predict('''
def process_matrix(matrix):
    result = []
    for row in matrix:
        row_result = []
        for cell in row:
            if cell is not None:
                if isinstance(cell, (int, float)):
                    row_result.append(cell * 2)
        result.append(row_result)
    return result
''', "Complex but correct (deeply nested)")

In [ ]:
# Semantic bug (logic error, hard to detect)
demo_predict('''
def is_prime(n):
    """Check if n is prime."""
    if n < 2:
        return False
    for i in range(2, n):  # Bug: should be range(2, int(n**0.5)+1)
        if n % i == 0:
            return False
    return True
''', "Semantic bug (inefficient but works)")

In [ ]:
# Race condition (impossible to detect statically)
demo_predict('''
counter = 0

def increment():
    global counter
    counter += 1  # Not thread-safe!
    return counter
''', "Race condition (hard to detect statically)")

## 5. Interactive Prediction

Try your own code snippets!

In [ ]:
# Try your own code here!
your_code = '''
def your_function():
    # Write your Python function here
    pass
'''

demo_predict(your_code, "Your Custom Code", show_features=True)

## 6. Batch Analysis

Analyze multiple functions at once.

In [ ]:
# Collection of test functions
test_functions = [
    ("def add(a,b): return a+b", "Simple add"),
    ("def div(a,b): return a/b", "Division (no check)"),
    ("def safe_div(a,b): return a/b if b else None", "Safe division"),
    ("def buggy(x=[]): x.append(1); return x", "Mutable default"),
    ("def fixed(x=None): x = x or []; return x", "Fixed default"),
]

print('Batch Analysis Results:')
print('='*70)
print(f'{"Function":<35} {"Prediction":<10} {"Probability":<12}')
print('-'*70)

for code, name in test_functions:
    result = predictor.predict(code)
    prob = result.get('probability', 0) or 0
    pred = result.get('prediction', 'ERROR')
    print(f'{name:<35} {pred:<10} {prob:.1%}')

print('='*70)

## Summary

This demo showed:

1. **Classic bugs**: Division by zero, index errors - often detected
2. **Python gotchas**: Mutable defaults, bare except - usually detected
3. **Clean code**: Well-structured functions - mostly classified correctly
4. **Edge cases**: Very short functions, semantic bugs - model may struggle

### Model Limitations

- **Semantic bugs**: Logic errors that are syntactically correct
- **Context-dependent bugs**: Require understanding of external state
- **Race conditions**: Concurrency issues
- **Very short functions**: Not enough features to analyze

### Tips for Best Results

1. Provide complete functions (not snippets)
2. Include docstrings and comments
3. Use meaningful variable names
4. Remember: the model provides probability, not certainty!